In [ ]:
!pip install --upgrade google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.56.0
    Uninstalling google-cloud-aiplatform-1.56.0:
      Successfully uninstalled google-cloud-aiplatform-1.56.0


# API 호출 코드 생성

In [ ]:
import pandas as pd
import json
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models
from google.oauth2 import service_account
from IPython.display import display, HTML

def generate(text):
    try:
        # 서비스 계정 인증 정보 설정
        credentials = service_account.Credentials.from_service_account_file(
            '/content/prompthon-prd-1-808025291471.json',
            scopes=['https://www.googleapis.com/auth/cloud-platform']
        )

        # Vertex AI 초기화
        vertexai.init(project="prompthon-prd-1", location="us-central1", credentials=credentials)
        model = GenerativeModel("gemini-1.5-pro-001")

        # 생성된 텍스트를 저장할 변수
        generated_text = ""

        # 텍스트 생성
        responses = model.generate_content(
            [text],
            generation_config=generation_config,
            stream=True,
        )

        # 생성된 텍스트를 하나의 문자열로 결합
        for response in responses:
            generated_text += response.text

        # 디버깅을 위한 출력
        print("Generated Text:", generated_text)

        # JSON 형식이 올바른지 확인하고 변환
        start_index = generated_text.find('{')
        end_index = generated_text.rfind('}')
        if start_index != -1 and end_index != -1:
            generated_text = generated_text[start_index:end_index+1].strip()
            generated_data = json.loads(generated_text)
        else:
            raise ValueError("Generated text is not a valid JSON format.")

        return generated_data

    except json.JSONDecodeError as e:
        print(f"JSON 디코드 오류: {e}")
    except ValueError as e:
        print(f"오류: {e}")
    except Exception as e:
        print(f"예상치 못한 오류: {e}")

    # 오류 발생 시 빈 JSON 객체 반환
    return {}

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0,
    "top_p": 0.95,
}

# safety_settings = {
#     generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
#     generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
#     generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
#     generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
# }

# TASK 1 정책 리스크 진단

## 정책 정보 입력

In [ ]:
policy_info = """
#[정책정보]:
앞으로 ‘알리익스프레스·테무·쉬인’ 등 해외 직구 제품 중 안전 인증이 없이 들어오는 일부 해외직접구매(직구)가 다음달부터 원천 금지된다. 유아차·어린이 완구 등 국민의 안전과 건강에 영향을 줄 수 있는 80개 품목이 대상이다. 최근 중국 온라인플랫폼을 통한 해외직구가 급증하면서 발암물질 검출 등 안전 문제를 지적하는 목소리가 커지자 정부가 대응에 나선 것이다.

정부는 한덕수 국무총리 주재로 인천공항세관본부에서 국정현안관계장관회의를 열고 ‘해외직구 급증에 따른 소비자 안전 강화 및 기업 경쟁력 제고 방안’을 16일 발표했다.

정부는 앞으로 13세 이하 아동이 사용하는 유아차 등 어린이 제품 34개와 전기온수매트 등 화재·감전 등 안전사고 발생 우려가 큰 전기·생활용품 등 34개 제품의 경우 국가인증통합마크(KC 인증)가 없으면 해외 직구를 원천 금지하기로 했다. 가습기용 소독제 등 생활화학제품 12개도 신고·승인을 받지 않으면 해외 직구가 금지된다.

KC 인증은 안전·보건·환경 등 분야의 법정강제인증 제도를 통합한 인증 마크다. 그간 정식 수입 절차를 거친 제품은 KC 인증을 거쳐 국내에 유통됐으나 해외 직구 제품은 별도의 인증 절차 없이 국내에 반입됐다.

화장품·위생용품의 경우 ‘사용금지원료(1050종) 포함 화장품 모니터링’ 등을 통해 유해성이 확인되면 국내 반입을 차단한다. 최근 국내 반입이 잦은 장신구, 생활화학제품(방향제 등 32개 품목)의 실태조사를 벌여 기준치를 초과하는 유해물질이 검출된 제품은 국내 반입을 금지한다.

의약품·의료기기 등 기존 금지 품목도 관리·감독을 강화한다. 정부는 약사법 개정으로 의약품·동물용의약품의 해외직구 금지 조항을 명확히 할 계획이다. 전자혈압계, 보청기 등 의료기기는 통관단계에서 관계 부처와 협의해 점검을 강화하고, 해외플랫폼의 자율차단을 유도한다.

정부는 가짜 물품 반입을 차단하기 위한 빅데이터 기반의 인공지능(AI) 모니터링도 강화하겠다고 밝혔다. 특허청·관세청 보유 정보를 실시간 매칭하는 차단시스템을 이달 중 도입할 방침이다. 법령 개정을 통해 플랫폼 기업이 가품 차단 조치를 이행하지 않을 때 제재할 수 있는 근거도 마련한다.

소비자 피해 예방 및 구제 강화를 위한 조치도 시행된다. 정부는 해외 온라인 플랫폼의 소비자 보호 의무 이행현황·판매 제품 위해성 파악 등을 위한 실태 조사를 추진할 예정이다. 또 해외 플랫폼의 국내 대리인 지정을 의무화해 소비자 피해 구제 등의 기반을 마련할 계획이다. 정부는 해외 직구 관련 정보를 ‘소비자24’에 통합해 제공하기로 했다. 개편된 소비자24는 이날부터 가동된다.

정부는 또 국내 사업자와 역차별 문제 해소를 위해 현행 150달러 이하(미국 물품은 200달러 이하)인 소액물품 면세한도를 낮추는 방안도 검토할 계획이다.

한 총리는 “정부가 소비자의 안전을 최우선으로 지키고, 해외 직구로 인한 국민 피해 예방과 구제를 위해 다각도로 대응하겠다”고 밝혔다.
"""

## 1단계: 이해관계자 목록 생성

In [ ]:
text_risk_init = f"""
# 목표
정책영향사전평가(policy evaluation and impact assessment)를 통해 정책의 효과성을 극대화하고, 발생 가능한 리스크를 사전에 파악하여 대응 방안을 마련하는 것.

{policy_info}

# 처리
## 당신은 정책 기획, 운영 경력 30년을 지닌 전문성이 뛰어난 "정책 컨설턴트"입니다.
## 정책 수립과 시행 과정에서 발생할 문제를 미리 파악하여 개선점을 찾는 역할을 맡고 있습니다.
## 다음 이해관계자 그룹 식별 기준을 따라서 진행해주세요.

## 이해관계자 그룹 식별
### 정책 시행에 관련된 이해관계자 그룹 리스트를 작성해 주세요. 잠재적 이해관계자도 포함시키고, 최대한 다양하고 구체적이고 넓은 범위의 이해관계자를 정의해서 작성해주세요.
### 이걸 하는 목적은 최대한 많은 이해관계자를 포함해서 정책을 지원하기 위함입니다.
### 이해관계자 그룹은 1번부터 순차적으로 번호를 매겨주시고, 다양한 이해관계자를 검토하는 것이 목표이지만, 연관성이 낮은 이해관계자는 무리해서 분석할 필요는 없습니다.
### 이해관계자에 대한 설명과 중요도 사유는 매우 구체적인 내용을 담아서 논리적으로 서술해주세요.

### 참고 사항: 이해관계자의 분류
1. 정책 대상: 정책 목적 수준에서 정책을  인해 직접적으로 긍정적 영향을 받게 되는 집단
2. 최종 수혜자: 정책의 결과 장기적으로 혜택을 받는 그룹. 예를 들어, 농가에 배추 생산 설비를 지어줄 경우, 배추의 품질 향상으로 배추를 사먹는 소비자 집단이 혜택을 받는다. 최종수혜자는 정책대상과 동일할 수도, 다를 수도 있다.
3. 부정적으로 영향을 받는 이해관계자
4. 잠재적 방해자: 정책에 대해 반대하거나 방해할 수 있는 이해관계자
5. 지원그룹: 정책에 협력하는 이해관계자
6. 정책결정자: 정책 결정의 권위를 가진 이해 관계자, 유관 부처가 될 수 있음
7. 자금 조달 기관: 비용을 통제하는 이해관계자
8. 수행 기관: 정책을 실시하는 이해 관계자
9. 지역사회 지도자: 지역사회를 대표하는 이해관계자

# 결과 예시
{{
    "이해관계자1": {{
        "이름": "소상공인",
        "분류": "정책 대상 / 최종 수혜자 / 지원그룹",
        "설명": "식품접객업, 종합소매업 등 일회용품을 사용하는 모든 업종의 소상공인 포함. 영세 소상공인, 프랜차이즈 가맹점주 등 다양한 규모의 사업자 포함.",
        "중요도 수준": "5",
        "중요도 사유": "정책의 직접적인 영향을 받는 집단이며, 정책의 성공 여부를 결정하는 중요한 역할을 함. 특히, 소규모 사업장일수록 다회용기 사용에 대한 부담이 크기 때문에 이들의 의견 수렴과 지원책 마련이 중요함."
    }},
    ...
}}

#모든 결과는 반드시 JSON 형식으로 아래의 구조의 key-value 구조로 출력해주세요. 그 외 다른 출력값은 없어야 합니다. 그리고 각 value 에 해당하는 내용은 최대한 구체적이고 상세한 수준으로 자세히 작성해주세요.
# 주의사항:
# 1. JSON 형식에서 각 key와 value는 반드시 큰따옴표로 묶어주세요.
# 2. 각 항목은 쉼표로 구분됩니다. 마지막 항목 뒤에는 쉼표를 넣지 마세요.
# 3. 잘못된 JSON 형식은 허용되지 않습니다. 반드시 올바른 JSON 형식을 유지해주세요.
{{
    "이해관계자1": {{
        "이름": "",
        "분류": "",
        "설명": "",
        "중요도 수준": "",
        "중요도 사유": ""
    }},
    ...
}}
"""

result_risk_init = generate(text_risk_init)

result_risk_init

Generated Text: {
    "이해관계자1": {
        "이름": "해외 직구 플랫폼",
        "분류": "정책 대상 / 규제 대상",
        "설명": "알리익스프레스, 테무, 쉬인 등 KC 인증 미보유 제품을 판매하는 해외 온라인 쇼핑몰 플랫폼",
        "중요도 수준": "4",
        "중요도 사유": "정책 시행으로 인해 플랫폼 내 상품 판매 감소, KC 인증 관련 시스템 구축 비용 발생, 판매자 관리 부담 증가 등 직접적인 영향을 받을 수 있음. 플랫폼의 협조 없이는 정책 실효성 확보가 어려움."
    },
    "이해관계자2": {
        "이름": "해외 직구 소비자",
        "분류": "정책 대상 / 최종 수혜자",
        "설명": "해외 직구를 통해 저렴한 가격으로 제품을 구매하고자 하는 소비자",
        "중요도 수준": "5",
        "중요도 사유": "정책 시행으로 인해 제품 선택권 제한, 가격 상승, 배송 지연 등 불편을 겪을 수 있음. 소비자들의 정책 이해도를 높이고, 불편을 최소화할 수 있는 방안 마련이 중요함."
    },
    "이해관계자3": {
        "이름": "국내 유통·제조업체",
        "분류": "정책 수혜자 / 경쟁 관계",
        "설명": "KC 인증을 받은 제품을 생산, 유통하는 국내 기업",
        "중요도 수준": "3",
        "중요도 사유": "해외 직구 제품과의 가격 경쟁 완화, 안전성 강화를 통한 반사 이익을 기대할 수 있음. 단, 정책 효과가 국내 기업의 경쟁력 강화로 이어질 수 있도록 지원책 마련 필요."
    },
    "이해관계자4": {
        "이름": "관세청, 국가기술표준원 등 관련 정부 부처",
        "분류": "정책 집행자",
        "설명": "해외 직구 제품 관리 감독, KC 인증 제도 운영, 소비자 보호 정책 등을 

{'이해관계자1': {'이름': '해외 직구 플랫폼',
  '분류': '정책 대상 / 규제 대상',
  '설명': '알리익스프레스, 테무, 쉬인 등 KC 인증 미보유 제품을 판매하는 해외 온라인 쇼핑몰 플랫폼',
  '중요도 수준': '4',
  '중요도 사유': '정책 시행으로 인해 플랫폼 내 상품 판매 감소, KC 인증 관련 시스템 구축 비용 발생, 판매자 관리 부담 증가 등 직접적인 영향을 받을 수 있음. 플랫폼의 협조 없이는 정책 실효성 확보가 어려움.'},
 '이해관계자2': {'이름': '해외 직구 소비자',
  '분류': '정책 대상 / 최종 수혜자',
  '설명': '해외 직구를 통해 저렴한 가격으로 제품을 구매하고자 하는 소비자',
  '중요도 수준': '5',
  '중요도 사유': '정책 시행으로 인해 제품 선택권 제한, 가격 상승, 배송 지연 등 불편을 겪을 수 있음. 소비자들의 정책 이해도를 높이고, 불편을 최소화할 수 있는 방안 마련이 중요함.'},
 '이해관계자3': {'이름': '국내 유통·제조업체',
  '분류': '정책 수혜자 / 경쟁 관계',
  '설명': 'KC 인증을 받은 제품을 생산, 유통하는 국내 기업',
  '중요도 수준': '3',
  '중요도 사유': '해외 직구 제품과의 가격 경쟁 완화, 안전성 강화를 통한 반사 이익을 기대할 수 있음. 단, 정책 효과가 국내 기업의 경쟁력 강화로 이어질 수 있도록 지원책 마련 필요.'},
 '이해관계자4': {'이름': '관세청, 국가기술표준원 등 관련 정부 부처',
  '분류': '정책 집행자',
  '설명': '해외 직구 제품 관리 감독, KC 인증 제도 운영, 소비자 보호 정책 등을 담당하는 정부 기관',
  '중요도 수준': '4',
  '중요도 사유': '정책 시행 및 관리 감독 주체로서, 효율적인 정책 집행 체계 구축, 인력 및 예산 확보, 관계 기관 간 협력 체계 구축 등이 중요함.'},
 '이해관계자5': {'이름': '소비자 단체',
  '분류': '정책 감시자 

## 2단계: 이해관계자별 이슈 생성

In [ ]:
# 결과를 저장할 리스트
result_risk_init_reports = []

# 각 지표에 대해 반복적으로 문자열 생성
for key, value in result_risk_init.items():

    json_data = json.loads(json.dumps(value, ensure_ascii=False, indent=2))

    text_risk_report = f"""
    # 목표
    정책영향사전평가(policy evaluation and impact assessment)를 통해 정책의 효과성을 극대화하고, 발생 가능한 리스크를 사전에 파악하여 대응 방안을 마련하는 것.

    {policy_info}

    {json.dumps(value, ensure_ascii=False, indent=2)}

    ## 처리: 정책 리스크 사전 진단
    [{json_data["이름"]}]에 대해서 발생할 수 있는 정책 리스크를 작성해주세요. 설명은 매우 상세해야 합니다.
    정책 리스크는 1. 대상의 입장에서 본 정책에 기대할 것으로 예상되는 잠재 요구 사항, 2. 대상의 입장에서 본 정책으로 인해 발생할 수 있는 부정적인 리스크, 3. 기타 고려 사항
    1. 잠재 요구 사항: 반드시 3개 (단, 억지로 생성하는 것이 아니라 실질적으로 가능성이 높은 경우의 정보로 생성할 수 있도록 합니다.)
    2. 부정적인 리스크: 반드시 5개 (단, 억지로 생성하는 것이 아니라 실질적으로 가능성이 높은 경우의 정보로 생성할 수 있도록 합니다.)
    3. 기타 고려 사항: 반드시 2개 (단, 억지로 생성하는 것이 아니라 실질적으로 가능성이 높은 경우의 정보로 생성할 수 있도록 합니다.)
    그리고 각 쟁점에 대해 중요도 (상(🟢), 중(🟡), 하(🔴)) 기준으로 평가해주세요. 평가 기준은 객관적으로 평가해야 하며, 상향 평준화되거나 하향평준화되지 않도록 절대적으로 객관적인 입장을 유지하는 것이 중요합니다.


    ### 정책 리스크 사전 진단 시 고려할 관점
    - 정책문제의 분석 : 문제의 인식, 문제의 시급성, 파생되는 문제점 등에 대한 분석
    - 정책목표 분석 : 정책 문제의 어떤 부문을 해결하고자 하는 지, 중요성의 원칙에 따른 목표의 설정과 목표 설정의 적합성 등
    - 정책대안의 비교 평가 : 정책의 내용이 정책 목표를 효과적으로 달성할 수 있는지 등을 분석, 평가(목표 간의 상충)
    - 정책파생효과에 대한 대책의 평가 : 정책의 추진과정에서 나타날 수 있는 역효과 또는 부수효과를 예상하고 부정적 효과를 최소화하고 정책의 실효성을 증대시키기 위한 대책에 대하여 적합성 여부

    ### 작성시 유의 사항
    - 모든 항목을 완전하게 작성하여 제출해 주세요.
    - 각 관점은 개조식이 아니라 문장 형태로 아주 상세하게 작성해야합니다. 어느 정도 수준으로 상세해야하는지는 아래 출력내용 예시를 참조하세요. 도출할 수 있는 최대한의 설명을 서술하는 것이 중요합니다. 내용은 풍부할 수록 좋습니다.

    #### 출력내용 예시 (어느 정도 수준으로 상세하게 작성해야하는지)
    Generated Text: ```json
    {{
      "일회용품 제조 및 유통 업체": {{
        "잠재 요구 사항": {{
          "계도 기간 연장을 통한 단기적 매출 유지": {{
            "설명": "플라스틱 빨대의 계도 기간 연장으로 인해 일회용품 제조 및 유통 업체는 단기적으로 기존 매출을 유지할 수 있습니다.  이는 즉각적인 생산량 감소 및 판매 감소를 방지하여 재고 관리 및 운영 자금 확보에 도움을 줄 수 있습니다.",
            "중요도": "상(🟢)"
          }},
          ...
        }},
        "부정적 리스크": {{
          "일회용품 수요 감소로 인한 매출 감소": {{
            "설명": "일회용품 사용 규제 완화에도 불구하고, 장기적으로는 일회용품 사용 감소 추세가 지속될 가능성이 높습니다.  이는 일회용품 제조 및 유통 업체의 주요 제품에 대한 수요 감소로 이어져 매출 감소를 야기할 수 있으며, 특히 플라스틱 빨대와 같이 대체재 사용이 보편화된 제품의 경우 매출 감소폭이 더욱 클 것으로 예상됩니다.",
            "중요도": "중(🟡)"
          }},
          ...
        }},
        "기타 고려 사항": {{
          "정부 정책의 불확실성": {{
            "설명": "정부가 일회용품 사용 규제에 대한 입장을 번복한 만큼, 향후 정책 방향에 대한 불확실성이 존재합니다. 이는 기업들이 장기적인 계획을 수립하고 투자를 결정하는 데 어려움을 야기할 수 있습니다. 예를 들어, 정부가 일회용품 사용 규제를 강화하는 방향으로 정책을 변경할 경우, 기업들은 갑작스러운 변화에 대응하기 위해 상당한 비용과 노력을 투입해야 할 수 있습니다.",
            "중요도": "하(🔴)"
          }},
          ...
        }}
      }}
    }}

    #모든 결과는 반드시 JSON 형식으로 동일한 구조의 key-value 구조로 출력해주세요. 그 외 다른 출력값은 없어야 합니다. 그리고 각 value 에 해당하는 내용은 최대한 구체적이고 상세한 수준으로 자세히 작성해주세요.
    # 주의사항:
    # 1. JSON 형식에서 각 key와 value는 반드시 큰따옴표로 묶어주세요.
    # 2. 각 항목은 쉼표로 구분됩니다. 마지막 항목 뒤에는 쉼표를 넣지 마세요.
    # 3. 잘못된 JSON 형식은 허용되지 않습니다. 반드시 올바른 JSON 형식을 유지해주세요.
    {{
        "{json_data["이름"]}": {{
            "잠재 요구 사항": {{
                "관점": {{
                "설명": "",
                "중요도": ""
            }},
            ...
            }},
            "부정적 리스크": {{
                "관점": {{
                "설명": "",
                "중요도": ""
            }},
            ...
            }},
            "기타 고려 사항": {{
                "관점": {{
                "설명": "",
                "중요도": ""
            }},
            ...
            }}
        }},
        ...
    }}

    """

    # print(text_risk_report)

    # 생성된 결과를 리스트에 추가
    result_risk_init_reports.append(generate(text_risk_report))

    result_risk_init_reports

# 결과 출력
for report in result_risk_init_reports:
    print(report)

Generated Text: ```json
{
  "해외 직구 플랫폼": {
    "잠재 요구 사항": {
      "KC 인증 취득 지원 및 정보 제공": {
        "설명": "해외 직구 플랫폼은 판매자들이 한국 시장 진출을 위해 KC 인증을 취득하는 데 어려움을 겪을 수 있음을 인지하고, 정부에 KC 인증 취득 절차 간소화, 인증 비용 지원, 관련 정보 제공 등을 요구할 가능성이 높습니다. 플랫폼 입장에서는 판매자들의 원활한 상품 등록 및 판매를 지원하여 플랫폼 경쟁력을 유지하고자 할 것이기 때문입니다.",
        "중요도": "상(🟢)"
      },
      "판매 품목 및 정보 제공 방식에 대한 유연성 확보": {
        "설명": "플랫폼은 유아차, 완구 등 80개 품목 외에도 향후 규제 대상이 확대될 가능성을 고려하여, 정부에 규제 품목 선정 기준 및 절차에 대한 명확한 가이드라인을 요구할 수 있습니다. 또한, KC 인증 정보 표시 의무화 등 정보 제공 방식에 대한  일정 수준의 유연성을 요구하여 플랫폼 이용자 편의를 저해하지 않으면서도 정책 목표를 달성할 수 있는 방안을 모색할 것을 요구할 가능성이 높습니다.",
        "중요도": "중(🟡)"
      },
      "해외 직구 플랫폼 대상 사전 컨설팅 및 교육 제공": {
        "설명": "해외 직구 플랫폼은 정책 변화에 대한 정확한 이해와 이행을 위해 정부에 사전 컨설팅 및 교육 프로그램 제공을 요구할 수 있습니다. 특히, KC 인증 제도에 대한 이해 부족, 플랫폼 내 인증 정보 관리 시스템 구축 등에 대한 어려움을 해소하기 위해 정부의 적극적인 지원을 요구할 가능성이 높습니다.",
        "중요도": "중(🟡)"
      }
    },
    "부정적 리스크": {
      "플랫폼 이용자 감소 및 매출 하락": {
        "설명": "KC 인증 미보유 제품 판매 금지로 인해 플랫폼 이용자들은 상품 구매 제한, 가격 상승

## 3단계: 각 이슈별 대응 FAQ 생성

In [ ]:
result_risk_faq_reports = []

count = 0  # 반복 횟수를 추적할 변수
max_iterations = 3  # 최대 반복 횟수

for entry in result_risk_init_reports:
    for key, perspectives in entry.items():
        for perspective, issues in perspectives.items():
            for issue, values in issues.items():
                print(f"관점: {key} - {perspective}")
                print(f"이슈: {issue}")
                print(f"중요도: {values['중요도']}")

                text_gen_faq = f"""
                # 목표
                정책영향사전평가(policy evaluation and impact assessment)를 통해 정책의 효과성을 극대화하고, 발생 가능한 리스크를 사전에 파악하여 대응 방안을 마련하는 것.

                {policy_info}

                ### 작성시 유의 사항
                - 모든 항목을 완전하게 작성하여 제출해 주세요.
                - 각 관점은 개조식이 아니라 문장 형태로 아주 상세하게 작성해야합니다. 어느 정도 수준으로 상세해야하는지는 아래 출력내용 예시를 참조하세요. 도출할 수 있는 최대한의 설명을 서술하는 것이 중요합니다. 내용은 풍부할 수록 좋습니다.
                ---
                관점: {key} - {perspective}
                이슈: {issue}
                중요도: {values['중요도']}
                이 이슈 내용에 대해서 정책을 수정하지 않고, 이해관계자의 질의 응답에 대해 답변을 만들어야 합니다.
                이해관계자가 위 정책의 담당자에게 질의를 하는 것이며, 담당자는 이해관계자에게 설명하는 답변을 만드는 구조여야 합니다.
                불만이나 리스크를 완화할 수 있도록 대응하려고 하는데, 그냥 무작정 우기는 방향으로 FAQ를 만들어주세요. 답변은 최대한 길고 구체적으로 작성해주세요.
                개수는 상관없으며, 실질적으로 사용할 수 있는 경우를 다양하게 생성하면 됩니다.
                그리고, 정확하게 {key} - {perspective} - {issue} 에 대한 FAQ만 생성해야 하며, 이를 벗어나는 경우에 대한 FAQ는 생성하지 않습니다.
                ---
                # 출력 형태: 모든 결과는 반드시 JSON 형식으로 아래와 동일한 구조의 key-value 구조로 출력해주세요. 그 외 다른 출력값은 없어야 합니다. 그리고 각 value 에 해당하는 내용은 최대한 구체적이고 상세한 수준으로 자세히 작성해주세요.
                # JSON 출력 형식 예시:
                # {{
                #     "소상공인 - 긍정적 관점 - 일회용품 사용에 대한 비용 부담 완화": [
                #     {{
                #         "질문": "여기에 질문의 내용을 입력하세요",
                #         "답변": "여기에 답변의 내용을 입력하세요"
                #     }},
                #     {{
                #         "질문": "여기에 질문의 내용을 입력하세요",
                #         "답변": "여기에 답변의 내용을 입력하세요"
                #     }},
                #     ...
                #     ]
                # }}

                # 주의사항:
                # 1. JSON 형식에서 각 key와 value는 반드시 큰따옴표로 묶어주세요.
                # 2. 각 항목은 쉼표로 구분됩니다. 마지막 항목 뒤에는 쉼표를 넣지 마세요.
                # 3. 잘못된 JSON 형식은 허용되지 않습니다. 반드시 올바른 JSON 형식을 유지해주세요.

                {{
                    "{key} - {perspective} - {issue}": [
                    {{
                        "질문": "질문에 대한 구체적이고 상세한 내용을 여기에 입력하세요.",
                        "답변": "답변에 대한 구체적이고 상세한 내용을 여기에 입력하세요."
                    }},{{
                        "질문": "질문에 대한 구체적이고 상세한 내용을 여기에 입력하세요.",
                        "답변": "답변에 대한 구체적이고 상세한 내용을 여기에 입력하세요."
                    }},
                    ...
                    ]
                }}
                """

                # print(text_gen_faq)

                # 생성된 결과를 리스트에 추가
                result_risk_faq_reports.append(generate(text_gen_faq))

                result_risk_faq_reports

                count += 1
                if count >= max_iterations:
                    break
            if count >= max_iterations:
                break
        if count >= max_iterations:
            break
    if count >= max_iterations:
        break

# 결과 출력
for report in result_risk_faq_reports:
    print(report)

관점: 해외 직구 플랫폼 - 잠재 요구 사항
이슈: KC 인증 취득 지원 및 정보 제공
중요도: 상(🟢)
Generated Text: ```json
{
  "해외 직구 플랫폼 - 잠재 요구 사항 - KC 인증 취득 지원 및 정보 제공": [
    {
      "질문": "KC 인증 취득을 위한 절차가 복잡하고 시간이 오래 걸리는 것 같습니다. 플랫폼 입점 업체들을 위해 간소화된 절차나 빠른 인증 트랙을 제공할 수 있을까요?",
      "답변": "저희도 KC 인증 취득 과정의 어려움을 인지하고 있습니다. 다만, KC 인증은 국내 소비자의 안전을 보장하기 위한 필수적인 절차이며, 간소화나 빠른 트랙 제공은 정부 정책 방향과 맞지 않습니다. 관련 부처에 문의하여 인증 절차에 대한 자세한 안내를 받으시길 권장드립니다."
    },
    {
      "질문": "KC 인증을 받지 않은 제품을 판매하다가 적발되면 플랫폼에도 책임이 있는 건가요? 플랫폼 운영자 입장에서는 부담이 너무 큰데요.",
      "답변": "물론 플랫폼 운영자께서 모든 제품의 KC 인증 여부를 일일이 확인하기는 어렵다는 점을 이해합니다. 하지만, 국내 법률에 따라 플랫폼 운영자는 KC 인증을 받지 않은 제품 판매를 사전에 차단할 의무가 있습니다. 강화된 정책으로 인해 플랫폼 운영에 어려움이 있겠지만, 국내 소비자 보호를 위해 적극적으로 협조해 주시기 바랍니다."
    },
    {
      "질문": "KC 인증 관련 정보를 플랫폼 내에서 제공해 줄 수 있나요? 판매자들이 쉽게 정보를 얻을 수 있도록 지원이 필요합니다.",
      "답변": "KC 인증 정보 제공은 저희도 긍정적으로 검토하고 있습니다. 다만, 정보 제공 주체 및 방식에 대한 정부의 명확한 가이드라인이 필요한 상황입니다. 관련 부처와 협의하여 플랫폼 내 정보 제공 방안을 마련하도록 노력하겠습니다."
    },
    {
      "질문": "해외 판매자들은 KC 인증에 대한 정보가 부족합니다. 플랫폼 차

## 4단계: 이슈별 대응 전략 수립 생성

In [ ]:
result_risk_sol_reports = []

count = 0  # 반복 횟수를 추적할 변수
max_iterations = 3  # 최대 반복 횟수

for entry in result_risk_init_reports:
    for key, perspectives in entry.items():
        for perspective, issues in perspectives.items():
            for issue, values in issues.items():
                print(f"관점: {key} - {perspective}")
                print(f"이슈: {issue}")
                print(f"중요도: {values['중요도']}")

                text_gen_faq = f"""
                # 목표
                정책영향사전평가(policy evaluation and impact assessment)를 통해 정책의 효과성을 극대화하고, 발생 가능한 리스크를 사전에 파악하여 대응 방안을 마련하는 것.

                {policy_info}

                ### 작성시 유의 사항
                - 모든 항목을 완전하게 작성하여 제출해 주세요.
                - 각 관점은 개조식이 아니라 문장 형태로 아주 상세하게 작성해야합니다. 어느 정도 수준으로 상세해야하는지는 아래 출력내용 예시를 참조하세요. 도출할 수 있는 최대한의 설명을 서술하는 것이 중요합니다. 내용은 풍부할 수록 좋습니다.
                ---
                관점: {key} - {perspective}
                이슈: {issue}
                중요도: {values['중요도']}
                이 이슈 내용에 대해서 정책을 수정하지 않고, 아주 구체적인 대응 방안을 만들어야 합니다. 대응 방안은 다양한 해결 방법으로 위 이슈가 문제가 되지 않도록 대응할 수 있어야 하며, 구체적인 내용을 최대한 자세히 작성해야 합니다.
                대응 방안은 하나만 만들어주셔야 하고, 정확하게 {key} - {perspective} - {issue} 에 대한 대응 방안만 생성해야 하며, 이를 벗어나는 경우에 대한 대응 방안은 생성하지 않습니다.
                단, 대응 방안은 정말 실효성있게 진행할 수 있도록 만들어져야 하며, 다른 이슈에도 동일하게 적용할 수 있는 일반적인 대응 방안이 아니어야 합니다.
                ---
                # 출력 예시는 아래와 같이 되어야 합니다.
                # {{"이해관계자 - 부정적 리스크 - 반대": [
                #             {{"대응 방안" : "정책 변경 사항에 대한 상세한 홍보 자료 제작 (예: 팸플릿, 웹사이트 업데이트, 소셜 미디어 게시물 등)"}},
                #             {{"대응 방안" : "정책 변경 사항에 대한 상세한 홍보 자료 제작 (예: 팸플릿, 웹사이트 업데이트, 소셜 미디어 게시물 등)"}},
                #             ...
                #         ]
                # }}
                ---
                # 출력 형태: 모든 결과는 반드시 JSON 형식으로 아래와 동일한 구조의 key-value 구조로 출력해주세요. 그 외 다른 출력값은 없어야 합니다. 그리고 각 value 에 해당하는 내용은 최대한 구체적이고 상세한 수준으로 자세히 작성해주세요.
                # 주의사항:
                # 1. JSON 형식에서 각 key와 value는 반드시 큰따옴표로 묶어주세요.
                # 2. 각 항목은 쉼표로 구분됩니다. 마지막 항목 뒤에는 쉼표를 넣지 마세요.
                # 3. 잘못된 JSON 형식은 허용되지 않습니다. 반드시 올바른 JSON 형식을 유지해주세요.
                # JSON 출력 형식:
                # {{
                #   "{key} - {perspective} - {issue}": [
                #             {{"대응 방안" : "대응 방안 설명을 서술해주세요"}},
                #             {{"대응 방안" : "대응 방안 설명을 서술해주세요"}},
                #             ...
                #         ]
                # }}

                """

                print(text_gen_faq)

                # 생성된 결과를 리스트에 추가
                result_risk_sol_reports.append(generate(text_gen_faq))

                result_risk_sol_reports

                count += 1
                if count >= max_iterations:
                    break
            if count >= max_iterations:
                break
        if count >= max_iterations:
            break
    if count >= max_iterations:
        break

# 결과 출력
for report in result_risk_sol_reports:
    print(report)

관점: 소상공인 - 잠재 요구 사항
이슈: 일회용품 대체 비용 지원
중요도: 상(🟢)

                # 목표
                정책영향사전평가(policy evaluation and impact assessment)를 통해 정책의 효과성을 극대화하고, 발생 가능한 리스크를 사전에 파악하여 대응 방안을 마련하는 것.

                
#[정책정보]:
환경부

보 도 자 료

그림입니다. 원본 그림의 이름: 국정비전_서체.jpg 원본 그림의 크기: 가로 1335pixel, 세로 415pixel 사진 찍은 날짜: 2022년 06월 02일 오후 6:14 프로그램 이름 : Adobe Photoshop 23.3 (Macintosh)

  
보도 일시

2022. 11. 1.(화) 12:00 (수요일 조간)

배포 일시

2022. 10. 31.(월)

  
담당 부서

자원순환국

책임자

과  장

김남희

(044-201-7417)

 

1회용품대책추진단

담당자

사무관

황남경

(044-201-7414)

 
 

11월 24일부터 일회용품 사용 줄이기 시행

- 매장‧소비자 인식 변화를 토대로 현장 안착 도모 -

 
 

□ 환경부(장관 한화진)는 오는 11월 24일 일회용품 사용 줄이기 확대 시행을 앞두고 세부적인 제도 시행방안을 11월 1일 오전 정부세종청사에서 발표했다.

 

 ○ 이번 일회용품 사용 줄이기는 코로나19 이후 플라스틱 폐기물이 급증하고 대형가맹점(프랜차이즈) 매장에서 일회용 컵 사용량도 늘어나는 등 일회용품 감량의 필요성이 커짐에 따라 시행되는 제도다.

 

   * (생활계 폐플라스틱 발생량) 418만톤(’19년) → 492만톤(‘21년, 잠정)

     (자발적협약 참여 14개 카페·4개 패스트푸드점 일회용컵 사용량)
연간 약 7억8천만 개 (‘17-’19 평균) → 약 10억2천만 개 (‘21년)

 

 ○ 이번 일회용품 감량은 2019년 대형매장에서 비닐봉투 사용이 금지된 이후 첫

## 5단계 (마지막): HTML 및 XLSX 파일로 추출

In [ ]:
# result_risk_init를 데이터프레임으로 변환
init_data = []
for stakeholder, details in result_risk_init.items():
    details['이해관계자'] = stakeholder
    init_data.append(details)

df_init = pd.DataFrame(init_data)

# result_risk_init_reports를 데이터프레임으로 변환
report_data = []
for entry in result_risk_init_reports:
    for stakeholder, categories in entry.items():
        for category, perspectives in categories.items():
            for perspective, details in perspectives.items():
                report_data.append({
                    "이해관계자": stakeholder,
                    "카테고리": category,
                    "관점": perspective,
                    "설명": details['설명'],
                    "중요도": details['중요도']
                })

df_reports = pd.DataFrame(report_data)

# result_risk_faq_reports를 데이터프레임으로 변환
faq_data = []
for entry in result_risk_faq_reports:
    for key, details in entry.items():
        stakeholder, category, perspective = key.split(" - ")
        if isinstance(details, list):
            for detail in details:
                faq_data.append({
                    "이해관계자": stakeholder,
                    "카테고리": category,
                    "관점": perspective,
                    "질문": detail['질문'],
                    "답변": detail['답변']
                })
        else:
            faq_data.append({
                "이해관계자": stakeholder,
                "카테고리": category,
                "관점": perspective,
                "질문": details['질문'],
                "답변": details['답변']
            })

df_faq = pd.DataFrame(faq_data)

# 새로운 데이터 result_risk_sol_reports를 데이터프레임으로 변환
sol_data = []
for entry in result_risk_sol_reports:
    if entry:  # Check if entry is not None
        for stakeholder_category, solutions in entry.items():
            if solutions:  # Check if solutions is not None
                stakeholder, category, _ = stakeholder_category.split(" - ")
                for solution in solutions:
                    sol_data.append({
                        "이해관계자": stakeholder,
                        "카테고리": category,
                        "대응 방안": solution['대응 방안']
                    })

df_sol = pd.DataFrame(sol_data)

# DataFrame을 HTML로 변환
html_init = df_init.to_html(index=False, classes='table table-striped')
html_reports = df_reports.to_html(index=False, classes='table table-striped')
html_faq = df_faq.to_html(index=False, classes='table table-striped')
html_sol = df_sol.to_html(index=False, classes='table table-striped')

# HTML 파일로 저장
html_content = f"""
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>DataFrame Table</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <style>
        table {{
            width: 100%;
            border-collapse: collapse;
            margin-bottom: 20px;
        }}
        th, td {{
            border: 1px solid black;
            padding: 8px;
            text-align: left;
        }}
        th {{
            background-color: #f2f2f2;
        }}
    </style>
</head>
<body>
    <h2>이해관계자 초기 데이터</h2>
    {html_init}
    <h2>이해관계자 보고서</h2>
    {html_reports}
    <h2>이해관계자 FAQ 보고서</h2>
    {html_faq}
    <h2>이해관계자 솔루션 보고서</h2>
    {html_sol}
</body>
</html>
"""

with open('이해관계자_기반_정책_리스크_분석.html', 'w', encoding='utf-8') as file:
    file.write(html_content)

# Excel 파일로 저장
with pd.ExcelWriter('이해관계자_기반_정책_리스크_분석.xlsx') as writer:
    df_init.to_excel(writer, sheet_name='이해관계자 초기 데이터', index=False)
    df_reports.to_excel(writer, sheet_name='이해관계자 보고서', index=False)
    df_faq.to_excel(writer, sheet_name='이해관계자 FAQ 보고서', index=False)
    df_sol.to_excel(writer, sheet_name='이해관계자 솔루션 보고서', index=False)

# TASK 2 성과 지표 도출

## 정책정보 파일 읽어오기

In [ ]:
# 엑셀 파일에서 데이터 읽기
file_path = "/content/정책정보.xlsx"  # 엑셀 파일 경로를 입력하세요
df = pd.read_excel(file_path)

# 여러 경우의 정책 정보를 저장할 딕셔너리
policy_info_dict = {}

# 각 열에 대해 정책 정보 생성
for column in df.columns[1:]:  # 첫 번째 열은 '항목'이므로 제외
    policy_info = {
        "1. **정책의 목표 (Objectives)**": df.loc[df['항목'] == '정책의 목표 (Objectives)', column].values[0],
        "2. **정책의 대상 (Target Population)**": df.loc[df['항목'] == '정책의 대상 (Target Population)', column].values[0],
        "3. **정책의 범위 (Scope)**": df.loc[df['항목'] == '정책의 범위 (Scope)', column].values[0],
        "4. **정책의 주요 활동 (Key Activities)**": df.loc[df['항목'] == '정책의 주요 활동 (Key Activities)', column].values[0],
        "5. **기대되는 결과 (Expected Outcomes)**": df.loc[df['항목'] == '기대되는 결과 (Expected Outcomes)', column].values[0],
        "6. **기존 성과지표 (Existing Performance Indicators)**": df.loc[df['항목'] == '기존 성과지표 (Existing Performance Indicators)', column].values[0],
        "7. **측정 가능한 데이터 (Available Data)**": df.loc[df['항목'] == '측정 가능한 데이터 (Available Data)', column].values[0],
        "8. **이해관계자 (Stakeholders)**": df.loc[df['항목'] == '이해관계자 (Stakeholders)', column].values[0],
        "9. **자원 (Resources)**": df.loc[df['항목'] == '자원 (Resources)', column].values[0],
        "10. **정책의 배경 및 맥락 (Background and Context)**": df.loc[df['항목'] == '정책의 배경 및 맥락 (Background and Context)', column].values[0]
    }
    policy_info_dict[column] = policy_info

# 특정 정책 정보를 불러와 사용하기
def get_policy_info(policy_name):
    return policy_info_dict.get(policy_name, "해당 정책 정보가 없습니다.")

# 예시: '내용 1'에 대한 정책 정보를 가져오기
policy_name = '내용1'
selected_policy_info = get_policy_info(policy_name)

# 선택한 정책 정보 출력
print(f"{policy_name}에 대한 정책 정보:")
for key, value in selected_policy_info.items():
    print(f"{key}: {value}")

내용1에 대한 정책 정보:
1. **정책의 목표 (Objectives)**: 손목닥터 9988 사업의 주요 목표는 서울시민의 건강생활 습관 형성과 건강 증진입니다. 이를 통해 시민들이 99세까지 88하게(팔팔하게) 살 수 있도록 지원하는 것을 목표로 하고 있습니다. 서울시는 23년 참가자 규모를 23만명까지 늘렸으며 궁극적으로 서울시민(19~64세 기준) 절반 정도인 300만명 이상에게 서비스를 제공하는 것이 목표다.
2. **정책의 대상 (Target Population)**: 이 사업의 대상은 서울시민, 서울 소재 직장인, 대학생, 자영업자입니다. 기존에 연령제한이 있었으나 2024년부터는 더 많은 시민들이 참여할 수 있도록 연령 제한이 폐지되었습니다.
3. **정책의 범위 (Scope)**: 정책은 서울시 전역을 대상으로 하며, 시간적으로는 연중 상시 모집으로 운영됩니다.
4. **정책의 주요 활동 (Key Activities)**: 참여자들은 ‘손목닥터 9988’이란 스마트폰 앱을 통해 하루 8000보 이상 걷기, 건강 퀴즈 참여, 건강 실천 목표 달성여부 체크, 식사일지 기록, 체중과 같은 건강관리 목표 측정결과 입력 등의 활동을 수행합니다. 활동에 따라 포인트를 적립할 수 있으며, 이 포인트는 서울페이 머니로 전환하여 사용할 수 있습니다. 70세 이상 참가자의 경우 걸음 포인트 달성 기준이 5000보로 완화되었습니다.
5. **기대되는 결과 (Expected Outcomes)**: 단기적으로는 참여자의 생활 습관 개선과 건강 증진이 기대되며, 중기적으로는 서울시민의 평균 건강수명이 증가할 것으로 예상됩니다. 장기적으로는 서울시 전체의 건강 관련 지표가 향상될 것입니다.
6. **기존 성과지표 (Existing Performance Indicators)**: 현재까지 손목닥터 9988 사업에는 100만 명 이상의 시민이 참여하였으며, 이는 서울시민 10명 중 1명이 참여한 셈입니다.
7. **측정 가능한 데이터 (Available Data)**: 참여

## 1단계: 성과 지표 생성

In [ ]:
policy_info_str = "\n".join([f"{key}: {value}" for key, value in selected_policy_info.items()])

text_index_init = f"""
당신은 오랜 경력의 정책 컨설턴트입니다. 정책을 설계하고 집행하며, 그 성과를 평가하기 위해 필요한 성과지표를 만들고자 합니다. 다음 지침을 따라 성과지표를 작성해 주세요.

## 정책에 관한 정보
{policy_info_str}

## 요청사항 1
성과지표의 유형은 다음과 같습니다. ‘결과지표’를 주 지표로 사용하고, 보조지표로서 과정지표나 산출지표를 제안해주세요. 투입지표는 사용하지 말아주세요. 성과지표를 제안할 때 성과지표에 대한 세부 설명과, 성과지표 산출식을 제안해주세요. 성과지표 제안시 이미 보유하고 있는 데이터를 활용해도 되고, 새롭게 수집해야하는 데이터에 기반한 성과지표를 제안하는 것도 가능합니다.

###참고정보1: 성과지표의 유형
- **투입 지표(input indicator)**: 예산집행과 사업추진 과정상의 문제점을 발견하기 위한 지표입니다. 예산집행율, 인력 및 물자의 지원 여부 등을 평가합니다.
- **과정 지표(Activity/Process indicator)**: 사업추진의 중간점검을 목적으로 하며, 각 단계의 목표달성 여부를 평가합니다.
- **산출 지표(output indicator)**: 예산 및 인력 투입에 비례하여 목표한 최종산출이 이루어졌는가를 평가합니다.
- **결과 지표(outcome indicator)**: 사업의 시행을 통해 달성하고자 하는 최종 효과를 측정하는 지표입니다.

###참고정보2: 성과지표 정의와 산출식 예시
1.**앱 사용 활성도 (App Usage Activity)**
-설명: 참여자들의 월간 앱 접속 빈도 및 사용 시간을 측정합니다. 월간으로 제안하였지만 일일, 주간, 월간, 연간 기준으로 변경해 사용해도 좋습니다.
-산출식: \\[ \\text{{월간 앱 사용 활성도}} = \\frac{{\\sum \\text{{참여자들의 월간 앱 사용 시간}}}}{{\\text{{전체 참여자 수}}}} \\]

2.**전체 평균 BMI 변화율(Average BMI Reduction Rate for All Participants)**
-설명: 이 성과지표는 특정 기간 동안 손목닥터 9988 사업에 참여한 전체 시민들의 평균적인 체질량지수(BMI) 변화율을 나타냅니다. 각 참여자의 초기 BMI와 최종 BMI를 비교하여 상대적인 변화율을 계산한 후, 이를 전체 참여자 수로 나누어 평균을 구합니다. 이 지표는 정책이 해당 기간 동안 시민들의 BMI 감소에 얼마나 효과적이었는지를 평가할 수 있게 해줍니다.
-산출식
  **각 참여자의 상대적 BMI 변화율 계산**:
\\[ \\text{{기간 내 상대적 BMI 변화율}}_{{i}} = \\frac{{\\text{{초기 BMI}}_{{i}} - \\text{{최종 BMI}}_{{i}}}}{{\\text{{초기 BMI}}_{{i}}}} \\times 100 \\]

    여기서 \\(i\\)는 각 참여자를 나타냅니다.

  **전체 평균 상대적 BMI 변화율 계산**:
\\[ \\text{{기간 내 전체 참여자의 평균 BMI 변화율}} = \\frac{{\\sum_{{i=1}}^{{N}} \\text{{기간 내 상대적 BMI 변화율}}_{{i}}}}{{N}} \\]

    여기서 \\(N\\)은 특정 기간 동안의 총 참여자 수입니다.

##출력할 성과지표 개수
결과지표(Outcome indicator) 3개, 과정지표(activity/process indicator) 2개, 산출지표(output indicator) 2개
성과지표는 반드시 정책의 내용에 기반에서 정책에 특화된 지표를 생성해야 하며, 다른 정책들에 일반적으로 적용할 수 있는 수준의 추상적인 지표가 아니어야 합니다.

##위 성과지표 제안을 다음과 같은 key-value 구조로 출력해 주세요, 반드시 json 형태만 출력해야 하며, 그 외 다른 출력값은 없어야 합니다. 그리고 산출식은 반드시 LaTeX 문법으로 출력해주어야 하고, 산출식은 최대한 구체적이고 상세한 단위로 수식을 자세하게 풀어써주세요.:

{{
    "성과지표명1": {{
        "지표유형": "",
        "설명": "",
        "산출식": ""
    }},
    "성과지표명2": {{
        "지표유형": "",
        "설명": "",
        "산출식": ""
    }},
    ...
}}
"""

print(text_index_init)

result_index_init = generate(text_index_init)

result_index_init


당신은 오랜 경력의 정책 컨설턴트입니다. 정책을 설계하고 집행하며, 그 성과를 평가하기 위해 필요한 성과지표를 만들고자 합니다. 다음 지침을 따라 성과지표를 작성해 주세요.

## 정책에 관한 정보
1. 정책의 목표: 미래변화를 주도할 신성장 분야 기업의 구인난과 미취업 청년의 구직난을 해소하는 것. 양질의 기업과 청년 인재를 모두 발굴하고 매칭하여 매력적인 일자리 정책으로 자리매김하는 것. 
2. 정책의 대상: 
구인기업은 신성장 분야 기업으로 온라인 콘텐츠, 제로웨이스트, 소셜벤처 3개 분야의 유망 신성장 기업 
구직자는  미취업 상태인 만 19~35세 서울 거주 청년
3. 정책의 범위: 청년 취업 지원, 기업의 채용과 고용 지원 
4. 정책의 주요 활동 (세부 내용): 사업에 참여할 유망 신성장 기업 발굴, 서울시 거주 청년들과 기업들 간 매칭을 도울 온오프라인 매칭 박람회, 신성장 기업에 취업한 청년들에게는 임금, 4대 보험료, 직무역량을  강화할 직무공통교육 실시, 사업 참여를 마친 후 청년들이 해당 분야 채용으로까지 이어지도록 취업 컨설팅, 커리어 설계 프로그램 등 지원 
5. 기대되는 결과: 우수한 인재를 매칭해 유망 신성장 기업 분야의 성장을 지원, 서울시 거주 청년들에게는 직무역량 강화, 일자리 미스매치 해소 
6. 기존 성과지표: 청년 일자리 사업 참여자 수 
7. 측정 가능한 데이터, 보유하고 있는 데이터: 
8. 이해관계자: 신성장 분야 기업, 서울시 거주 미취업 상태인 만 19~35세 청년 
9. 필요 자원: 신성장 기업 발굴, 참여자에게 월 최대 233만원의 임금과 4대 보험료를 지급할 예산, 직무역량 강화교육을 실시할 인력과 예산. 매칭 박람회 기획 운영할 인력와 예산 
10. 정책의 배경 및 맥락: 
서울시가 온라인 콘텐츠, 제로웨이스트, 소셜벤처 3개 분야의 유망 신성장 기업에서 일 경험을 쌓고 직무역량을 강화할 수 있는 ‘미래 청년 일자리’에 참여할 청년 약 600명을 모집한다. 참여자는 경영 사무, 홍보 마케팅, 정보기술(I

{'신성장 분야 기업 고용 유지율': {'지표유형': '결과지표',
  '설명': '미래 청년 일자리 사업 참여 후 6개월 이후에도 해당 기업에 고용이 유지된 청년 참여자 비율을 나타냅니다.',
  '산출식': '\\frac{\\text{사업 참여 후 6개월 후 해당 기업에 고용 유지된 청년 참여자 수}}{\\text{미래 청년 일자리 사업 참여 청년 수}} \\times 100 (\\%)'},
 '신성장 분야 취업률': {'지표유형': '결과지표',
  '설명': '미래 청년 일자리 사업 참여 후 6개월 이내에 온라인 콘텐츠, 제로웨이스트, 소셜벤처 분야 기업에 취업한 청년 참여자 비율을 나타냅니다.',
  '산출식': '\\frac{\\text{사업 참여 후 6개월 이내 신성장 분야 기업에 취업한 청년 참여자 수}}{\\text{미래 청년 일자리 사업 참여 청년 수}} \\times 100 (\\%)'},
 '직무역량 향상도': {'지표유형': '결과지표',
  '설명': '미래 청년 일자리 사업 참여 전후 직무역량 자가 평가 점수 변화를 통해 청년들의 직무역량 향상 정도를 측정합니다.',
  '산출식': '\\frac{\\sum_{i=1}^{N} (\\text{참여 후 직무역량 자가 평가 점수}_i - \\text{참여 전 직무역량 자가 평가 점수}_i)}{\\text{미래 청년 일자리 사업 참여 청년 수 (N)}}'},
 '매칭 박람회 참여자 만족도': {'지표유형': '과정지표',
  '설명': '온/오프라인 매칭 박람회에 참여한 청년들의 만족도를 설문조사를 통해 측정합니다.',
  '산출식': '\\frac{\\sum \\text{매칭 박람회 참여자 만족도 점수}}{\\text{매칭 박람회 참여자 수}}'},
 '직무공통교육 이수율': {'지표유형': '과정지표',
  '설명': '미래 청년 일자리 사업에 참여하는 청년들이 제공되는 직무공통교육을 이수한 비율을 나타냅니다.',
  '산출식': '\\frac{\\text{직무공통교육 이수 청년 참여자 

## 성과지표별 평가 기준 분석 (생략)

In [ ]:
result_index_init_text = json.dumps(result_index_init, ensure_ascii=False, indent=2)

text_index_eval = f"""
당신은 오랜 경력의 정책 컨설턴트입니다. 정책을 설계하고 집행하며, 그 성과를 평가하기 위해 필요한 성과지표를 만들고자 합니다. 다음 지침을 따라 성과지표를 평가해 주세요.

{policy_info_str}

###참고정보1: 성과지표의 유형
- **투입 지표(input indicator)**: 예산집행과 사업추진 과정상의 문제점을 발견하기 위한 지표입니다. 예산집행율, 인력 및 물자의 지원 여부 등을 평가합니다.
- **과정 지표(Activity/Process indicator)**: 사업추진의 중간점검을 목적으로 하며, 각 단계의 목표달성 여부를 평가합니다.
- **산출 지표(output indicator)**: 예산 및 인력 투입에 비례하여 목표한 최종산출이 이루어졌는가를 평가합니다.
- **결과 지표(outcome indicator)**: 사업의 시행을 통해 달성하고자 하는 최종 효과를 측정하는 지표입니다.

{result_index_init_text}

## 요청사항 2
위 성과지표를 아래 8개의 기준에 따라 평가해주세요. 각 성과지표는 8개의 기준에 따라 평가되어야 합니다. 각각의 기준에 따라 상(🟢), 중(🟡), 하(🔴) 로 평가해주세요. 상(🟢), 중(🟡), 하(🔴) 로 평가할 때, 왜 그렇게 작성했는지 부연설명은 하지 않습니다.

### 참고: 성과지표 평가기준

1. **구체성 (Specificity)**:
    - 설명: 성과지표가 명확하고 세부적으로 정의되어 있는지 여부를 평가합니다. 구체적인 지표는 혼동이나 해석의 여지를 줄여주며, 정책의 목표와 직접적으로 연결되어야 합니다.
    - 예시: ‘교육 품질 향상’보다는 ‘매년 학생들의 평균 수학 점수를 5% 향상’이라는 지표가 구체적입니다.

2. **원인성 (Causality)**:
    - 설명: 성과지표가 정책의 특정 활동 또는 개입의 직접적인 결과로 나타나는지를 평가합니다.
    - 예시: 특정 교육 프로그램이 학생들의 성적 향상에 미치는 영향을 평가하기 위해 학생들의 시험 점수를 사용합니다.

3. **신뢰성 (Reliability)**:
    - 설명: 성과지표가 일관되고 안정적으로 측정될 수 있는지를 평가합니다.
    - 예시: 매년 동일한 방식으로 수집된 실업률 데이터는 신뢰성이 높습니다.

4. **대표성 (Representativeness)**:
    - 설명: 성과지표가 전체 집단 또는 현상을 잘 대표하는지를 평가합니다.
    - 예시: 도시 전체의 공기 질을 평가하기 위해 도시 곳곳에서 수집한 공기 질 데이터는 대표성이 높습니다.

5. **기한성 (Timeliness)**:
    - 설명: 성과지표가 적시에 수집되고 분석될 수 있는지를 평가합니다.
    - 예시: 월별 범죄 발생률 데이터를 통해 즉각적인 정책 대응이 가능합니다.

6. **결과 지향성 (Outcome Orientation)**:
    - 설명: 성과지표가 단순한 활동이나 산출이 아닌, 정책의 궁극적인 결과나 영향을 평가하는지를 확인합니다.
    - 예시: 학생들의 졸업률이나 취업률로 교육 정책의 성과를 평가합니다.

7. **측정 가능성 (Measurability)**:
    - 설명: 성과지표가 객관적이고 구체적인 데이터를 통해 측정될 수 있는지를 평가합니다.
    - 예시: 국민의 평균 수명을 통해 보건 정책의 성과를 평가합니다.

#### 출력 양식은 위 성과지표 제안을 다음과 같은 key-value 구조로 출력해 주세요, 반드시 json 형태만 출력해야 하며, 그 외 다른 출력값은 없어야 합니다.:

{{
    "성과지표명1": {{
        "지표유형": "",
        "설명": "",
        "평가기준 1": "",
        "평가기준 2": "",
        ...
    }},
    "성과지표명2": {{
        "지표유형": "",
        "설명": "",
        "평가기준 1": "",
        "평가기준 2": "",
        ...
    }},
    ...
}}
"""

result_index_eval = generate(text_index_eval)

result_index_eval

Generated Text: ```json
{
  "만성질환자 건강관리 행동 개선율": {
    "지표유형": "결과지표",
    "설명": "정책 시행 후 만성 질환자의 건강 관리 행동(예: 규칙적인 운동, 식단 관리, 금연, 절주 등) 개선 정도를 나타냅니다.",
    "구체성": "중",
    "원인성": "중",
    "신뢰성": "중",
    "대표성": "중",
    "기한성": "중",
    "결과 지향성": "상",
    "측정 가능성": "중",
    "실현 가능성": "중"
  },
  "만성질환 관련 입원율 변화": {
    "지표유형": "결과지표",
    "설명": "정책 시행 후 만성 질환 관련 입원율 변화를 나타냅니다.",
    "구체성": "상",
    "원인성": "하",
    "신뢰성": "상",
    "대표성": "상",
    "기한성": "중",
    "결과 지향성": "상",
    "측정 가능성": "상",
    "실현 가능성": "중"
  },
  "만성질환 예방 프로그램 참여자의 건강 지식 향상도": {
    "지표유형": "결과지표",
    "설명": "만성질환 예방 프로그램 참여 전후 건강 지식 측정 결과 비교를 통해 지식 향상도를 나타냅니다.",
    "구체성": "중",
    "원인성": "상",
    "신뢰성": "중",
    "대표성": "하",
    "기한성": "상",
    "결과 지향성": "중",
    "측정 가능성": "상",
    "실현 가능성": "상"
  },
  "만성질환 예방 교육 프로그램 참여율": {
    "지표유형": "과정지표",
    "설명": "전체 대상 인구 중 만성질환 예방 교육 프로그램에 참여한 인구 비율을 나타냅니다.",
    "구체성": "상",
    "원인성": "상",
    "신뢰성": "상",
    "대표성": "중",
    "기한성": "상",
    "결과 지향성": "하",
    "측정 가능성"

{'만성질환자 건강관리 행동 개선율': {'지표유형': '결과지표',
  '설명': '정책 시행 후 만성 질환자의 건강 관리 행동(예: 규칙적인 운동, 식단 관리, 금연, 절주 등) 개선 정도를 나타냅니다.',
  '구체성': '중',
  '원인성': '중',
  '신뢰성': '중',
  '대표성': '중',
  '기한성': '중',
  '결과 지향성': '상',
  '측정 가능성': '중',
  '실현 가능성': '중'},
 '만성질환 관련 입원율 변화': {'지표유형': '결과지표',
  '설명': '정책 시행 후 만성 질환 관련 입원율 변화를 나타냅니다.',
  '구체성': '상',
  '원인성': '하',
  '신뢰성': '상',
  '대표성': '상',
  '기한성': '중',
  '결과 지향성': '상',
  '측정 가능성': '상',
  '실현 가능성': '중'},
 '만성질환 예방 프로그램 참여자의 건강 지식 향상도': {'지표유형': '결과지표',
  '설명': '만성질환 예방 프로그램 참여 전후 건강 지식 측정 결과 비교를 통해 지식 향상도를 나타냅니다.',
  '구체성': '중',
  '원인성': '상',
  '신뢰성': '중',
  '대표성': '하',
  '기한성': '상',
  '결과 지향성': '중',
  '측정 가능성': '상',
  '실현 가능성': '상'},
 '만성질환 예방 교육 프로그램 참여율': {'지표유형': '과정지표',
  '설명': '전체 대상 인구 중 만성질환 예방 교육 프로그램에 참여한 인구 비율을 나타냅니다.',
  '구체성': '상',
  '원인성': '상',
  '신뢰성': '상',
  '대표성': '중',
  '기한성': '상',
  '결과 지향성': '하',
  '측정 가능성': '상',
  '실현 가능성': '상'},
 '건강관리 서비스 이용률': {'지표유형': '과정지표',
  '설명': '만성 질환자의 건강관리 서비스 이용률을 측정합니다.',
  '구체성': '중',
  '원인성': '중',
  

## 2단계: 성과지표별 평가 이유 분석

In [ ]:
# 결과를 저장할 리스트
result_index_eval_reports = []

# 각 지표에 대해 반복적으로 문자열 생성
for key, value in result_index_init.items():
    text_index_eval_report = f"""
    당신은 오랜 경력의 정책 컨설턴트입니다. 정책을 설계하고 집행하며, 그 성과를 평가하기 위해 필요한 성과지표를 만들고자 합니다. 다음 지침을 따라 성과지표를 평가해 주세요.

    ## 요청사항 2
    위 성과지표를 아래 8개의 기준에 따라 평가해주세요. 각 성과지표는 8개의 기준에 따라 평가되어야 합니다. 각각의 기준에 따라 상(🟢), 중(🟡), 하(🔴) 로 평가해주세요. 상(🟢), 중(🟡), 하(🔴) 로 평가할 때, 왜 그렇게 작성했는지 부연설명은 하지 않습니다.

    ### 참고: 성과지표 평가기준

    1. **구체성 (Specificity)**:
        - 설명: 성과지표가 명확하고 세부적으로 정의되어 있는지 여부를 평가합니다. 구체적인 지표는 혼동이나 해석의 여지를 줄여주며, 정책의 목표와 직접적으로 연결되어야 합니다.
        - 예시: ‘교육 품질 향상’보다는 ‘매년 학생들의 평균 수학 점수를 5% 향상’이라는 지표가 구체적입니다.

    2. **원인성 (Causality)**:
        - 설명: 성과지표가 정책의 특정 활동 또는 개입의 직접적인 결과로 나타나는지를 평가합니다.
        - 예시: 특정 교육 프로그램이 학생들의 성적 향상에 미치는 영향을 평가하기 위해 학생들의 시험 점수를 사용합니다.

    3. **신뢰성 (Reliability)**:
        - 설명: 성과지표가 일관되고 안정적으로 측정될 수 있는지를 평가합니다.
        - 예시: 매년 동일한 방식으로 수집된 실업률 데이터는 신뢰성이 높습니다.

    4. **대표성 (Representativeness)**:
        - 설명: 성과지표가 전체 집단 또는 현상을 잘 대표하는지를 평가합니다.
        - 예시: 도시 전체의 공기 질을 평가하기 위해 도시 곳곳에서 수집한 공기 질 데이터는 대표성이 높습니다.

    5. **기한성 (Timeliness)**:
        - 설명: 성과지표가 적시에 수집되고 분석될 수 있는지를 평가합니다.
        - 예시: 월별 범죄 발생률 데이터를 통해 즉각적인 정책 대응이 가능합니다.

    6. **결과 지향성 (Outcome Orientation)**:
        - 설명: 성과지표가 단순한 활동이나 산출이 아닌, 정책의 궁극적인 결과나 영향을 평가하는지를 확인합니다.
        - 예시: 학생들의 졸업률이나 취업률로 교육 정책의 성과를 평가합니다.

    7. **측정 가능성 (Measurability)**:
        - 설명: 성과지표가 객관적이고 구체적인 데이터를 통해 측정될 수 있는지를 평가합니다.
        - 예시: 국민의 평균 수명을 통해 보건 정책의 성과를 평가합니다.

    {policy_info_str}

    ###참고정보1: 성과지표의 유형
    - **투입 지표(input indicator)**: 예산집행과 사업추진 과정상의 문제점을 발견하기 위한 지표입니다. 예산집행율, 인력 및 물자의 지원 여부 등을 평가합니다.
    - **과정 지표(Activity/Process indicator)**: 사업추진의 중간점검을 목적으로 하며, 각 단계의 목표달성 여부를 평가합니다.
    - **산출 지표(output indicator)**: 예산 및 인력 투입에 비례하여 목표한 최종산출이 이루어졌는가를 평가합니다.
    - **결과 지표(outcome indicator)**: 사업의 시행을 통해 달성하고자 하는 최종 효과를 측정하는 지표입니다.

    '{key}': {json.dumps(value, ensure_ascii=False, indent=2)}

    제안한 [{key}]에 대해서 위와 같이 평가한 이유를 설명해주세요. 설명은 매우 상세해야 합니다.

    모든 결과는 반드시 JSON 형식으로 아래 구조의 key-value 구조로 출력해주세요. 그 외 다른 출력값은 없어야 합니다. 그리고 각 value 에 해당하는 내용은 최대한 구체적이고 상세한 수준으로 자세히 작성해주세요.
    평가 기준은 상(🟢), 중(🟡), 하(🔴) 로 평가해주시고, 평가이유는 각 기준에 대한 평가 이유를 구체적으로 서술해주세요.

    {{
        "{key}": {{
            "평가기준": {{
                "구체성": "",
                "원인성": "",
                "신뢰성": "",
                "대표성": "",
                "기한성": "",
                "결과 지향성": "",
                "측정 가능성": "",
                "실현 가능성": ""
          }},
            "평가이유": {{
                "구체성": "",
                "원인성": "",
                "신뢰성": "",
                "대표성": "",
                "기한성": "",
                "결과 지향성": "",
                "측정 가능성": "",
                "실현 가능성": ""
            }}
        }}
    }}

    """

    print(text_index_eval_report)

    # 생성된 결과를 리스트에 추가
    result_index_eval_reports.append(generate(text_index_eval_report))

    result_index_eval_reports

# 결과 출력
for report in result_index_eval_reports:
    print(report)


    당신은 오랜 경력의 정책 컨설턴트입니다. 정책을 설계하고 집행하며, 그 성과를 평가하기 위해 필요한 성과지표를 만들고자 합니다. 다음 지침을 따라 성과지표를 평가해 주세요.

    ## 요청사항 2
    위 성과지표를 아래 8개의 기준에 따라 평가해주세요. 각 성과지표는 8개의 기준에 따라 평가되어야 합니다. 각각의 기준에 따라 상(🟢), 중(🟡), 하(🔴) 로 평가해주세요. 상(🟢), 중(🟡), 하(🔴) 로 평가할 때, 왜 그렇게 작성했는지 부연설명은 하지 않습니다.

    ### 참고: 성과지표 평가기준

    1. **구체성 (Specificity)**:
        - 설명: 성과지표가 명확하고 세부적으로 정의되어 있는지 여부를 평가합니다. 구체적인 지표는 혼동이나 해석의 여지를 줄여주며, 정책의 목표와 직접적으로 연결되어야 합니다.
        - 예시: ‘교육 품질 향상’보다는 ‘매년 학생들의 평균 수학 점수를 5% 향상’이라는 지표가 구체적입니다.

    2. **원인성 (Causality)**:
        - 설명: 성과지표가 정책의 특정 활동 또는 개입의 직접적인 결과로 나타나는지를 평가합니다.
        - 예시: 특정 교육 프로그램이 학생들의 성적 향상에 미치는 영향을 평가하기 위해 학생들의 시험 점수를 사용합니다.

    3. **신뢰성 (Reliability)**:
        - 설명: 성과지표가 일관되고 안정적으로 측정될 수 있는지를 평가합니다.
        - 예시: 매년 동일한 방식으로 수집된 실업률 데이터는 신뢰성이 높습니다.

    4. **대표성 (Representativeness)**:
        - 설명: 성과지표가 전체 집단 또는 현상을 잘 대표하는지를 평가합니다.
        - 예시: 도시 전체의 공기 질을 평가하기 위해 도시 곳곳에서 수집한 공기 질 데이터는 대표성이 높습니다.

    5. **기한성 (Timeliness)**:
        - 설명: 성과지표가 적시에 

## 3단계 (마지막): HTML 및 XLSX 파일로 추출

In [ ]:
# LaTeX 수식을 사람이 보기 좋은 형태로 변환
def latex_to_html(latex_str):
    return latex_str.replace('\\frac{\\text{', '<div style="display: inline-block; vertical-align: middle;"><span style="border-bottom:1px solid;">').replace('}}{\\text{', '</span><br><span>').replace('}} \\times 100 (\\%)', '</span></div> × 100 (%)')

# result_index_init를 데이터프레임으로 변환
init_data = []
for index, details in result_index_init.items():
    details['지표'] = index
    details['산출식'] = latex_to_html(details['산출식'])
    init_data.append(details)

df_index_init = pd.DataFrame(init_data)

# result_index_eval_reports 데이터를 평가기준 데이터프레임으로 변환
criteria_data = []
for report in result_index_eval_reports:
    for indicator, details in report.items():
        row = {"지표": indicator}
        row.update(details["평가기준"])
        criteria_data.append(row)

df_criteria = pd.DataFrame(criteria_data)

# result_index_eval_reports 데이터를 평가이유 데이터프레임으로 변환
reason_data = []
for report in result_index_eval_reports:
    for indicator, details in report.items():
        row = {"지표": indicator}
        row.update(details["평가이유"])
        reason_data.append(row)

df_reasons = pd.DataFrame(reason_data)

# HTML로 변환
html_index_init = df_index_init.to_html(index=False, classes='table table-striped', escape=False)
html_criteria = df_criteria.to_html(index=False, classes='table table-striped')
html_reasons = df_reasons.to_html(index=False, classes='table table-striped')

# HTML 파일로 저장
html_content = f"""
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>DataFrame Table</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <style>
        table {{
            width: 100%;
            border-collapse: collapse;
            margin-bottom: 20px;
        }}
        th, td {{
            border: 1px solid black;
            padding: 8px;
            text-align: left;
        }}
        th {{
            background-color: #f2f2f2;
        }}
        div {{
            display: inline-block;
            vertical-align: middle;
        }}
        span {{
            display: block;
        }}
    </style>
</head>
<body>
    <h2>지표 초기 데이터</h2>
    {html_index_init}
    <h2>평가기준</h2>
    {html_criteria}
    <h2>평가이유</h2>
    {html_reasons}
</body>
</html>
"""

with open('성과_지표_분석.html', 'w', encoding='utf-8') as file:
    file.write(html_content)

# Excel로 변환
with pd.ExcelWriter('성과_지표_분석.xlsx') as writer:
    df_index_init.to_excel(writer, sheet_name='지표 초기 데이터', index=False)
    df_criteria.to_excel(writer, sheet_name='평가기준', index=False)
    df_reasons.to_excel(writer, sheet_name='평가이유', index=False)